In [1]:
import tensorflow as tf
import numpy as np
import tensorflow_probability as tfp
import sys
import scipy.io
sys.path.append("..")
import ODE_Dynamics as od
import Test_Likelihood as tl
import Positive_Symptom_fn as fn

In [2]:
data = scipy.io.loadmat('../data/test_data_simulation.mat')
x = tf.cast(data['data_simulation'],dtype = tf.float32)
test_data = np.reshape(
      x, x.shape)

In [3]:
test_data = tf.transpose(test_data)

In [4]:
vdyn_ode_fn = od.ViralDynamics

In [5]:
positive_fn = fn.proba_pos_sym(170306.4 * 1E-5).positive_fn
symptom_fn = fn.proba_pos_sym(170306.4 * 1E-5).symptom_fn

In [6]:
prob_s_ibar = 0.1

In [7]:
loglike = tl.loglik(test_data, vdyn_ode_fn, positive_fn, symptom_fn, prob_s_ibar, prob_fp=0.0, Epi_Model=od.SIR,
                 duration=149.0, Epi_cadence=0.5, Vir_cadence=0.0625)

In [8]:
mu_b, sigma_b = 5, 1
beta = np.random.normal(mu_b, sigma_b, 1)
L = 0.0025/beta
V0 = np.random.normal(1E3, 1E2, 1)
print(V0)
X0 = 1E6
Y0 = V0
par=tf.constant(np.array([[L,0.01,beta*1E-7,0.5,20,10, V0, X0, Y0]], dtype=np.float32))
vpar = par
pospar = par
sympar = par

[1235.2980662]


In [10]:
sample_size = 1000
k = 1
index = 1
mu_b, sigma_b = 5, 1
beta = np.random.normal(mu_b, sigma_b, 1)   #"rate at which virus infects host cells"
L = 0.0025/beta
par=np.array([[L,0.01,beta*1E-7,0.5,20.0,10.0]])
V0 = np.random.normal(1E3, 1E2, 1)
X0 = 1E6
Y0 = V0
init_state=(np.array([[V0,X0,Y0]], dtype=np.float32))

while index <= sample_size - 1:
    beta = np.random.normal(mu_b, sigma_b, 1)   #"rate at which virus infects host cells"
    L = 0.0025/beta
    par_new=np.array([[L,0.01,beta*1E-7,0.5,20.0,10.0]])
    par = np.concatenate((par, par_new), axis = 0)
    V0 = np.random.normal(1E3, 1E2, 1)
    X0 = 1E6
    Y0 = V0
    init_state_new=(np.array([[V0,X0,Y0]], dtype=np.float32))
    init_state = np.concatenate((init_state, init_state_new), 0)

    index +=1
        

vpar = tf.constant(par, dtype=tf.float32)
pospar = par
sympar = par
print(par.shape)

(1000, 6)


In [11]:
epipar = tf.constant(np.array([[1.8,  5.0E-08,0.1, 0.001,0.999]], dtype=np.float32))

In [12]:
ll = loglike.__call__(test_data,epipar, vpar, pospar, sympar)

(1, 149, 2384)
printing likelihood
tf.Tensor(
[  -20.491894   -14.618128   -32.29079    -44.475746   -32.766212
   -38.909687   -70.01538    -57.948116   -70.87103    -71.39753
   -84.641846   -65.9818     -79.30226   -132.12808   -106.67795
  -133.96422   -101.56172   -115.80438   -150.54736   -130.99556
  -187.13786   -195.40016   -189.76947   -212.24362   -256.38757
  -272.47784   -259.95062   -283.4947    -307.35904   -412.3321
  -385.4964    -402.99048   -428.15762   -446.04544   -593.3931
  -566.5797    -608.6704    -674.4681    -725.4795    -808.37244
  -876.39856  -1048.4169   -1126.5079   -1157.2126   -1268.9872
 -1154.3102   -1258.8586   -1446.6469   -1520.6249   -1636.7571
 -1720.5189   -1922.4547   -1974.3617   -1984.1564   -2147.1328
 -2294.288    -2648.9707   -2574.6428   -2941.9111   -2937.3186
 -3317.531    -3349.0037   -3689.2976   -3606.588    -4083.2732
 -4250.6475   -4275.4053   -4371.8467   -4522.6685   -4429.5005
 -4925.978    -4777.8813   -4893.6406   -5312.447  